In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly

import test
import mylayers
import util

# Used to save temporary variables
import pickle


## Read data

In [2]:
df_train = pd.read_csv("./data/mnist_train.csv", header=None)
df_valid = pd.read_csv("./data/mnist_test.csv", header=None)


In [3]:
df_train.shape

(60000, 785)

### Shuffle the data set

In [3]:
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)


### Seperate feature column and target columns

In [4]:
X_train = np.array(df_train.drop(0, axis=1), dtype="f")
Y_train = df_train.loc[:, 0]
X_valid = np.array(df_valid.drop(0, axis=1), dtype="f")
Y_valid = df_valid.loc[:, 0]


### One hot encoding Y

In [5]:
encoder = util.onehot_encoder(Y_train)
Y_train = encoder.encoding(Y_train)
Y_valid = encoder.encoding(Y_valid)


## Build layers

In [6]:
layers = []
# ---Input Layer---
layers.append(mylayers.InputLayer(X_train))
# ---Hidden Layer---
# initialize FCL with xaiver
fc_layer = mylayers.FullyConnectedLayer(X_train.shape[1], 10)
weights, biases = util.xaiver_init(X_train.shape[1], 10)
fc_layer.setWeights(weights)
fc_layer.setBiases(biases)
layers.append(fc_layer)
layers.append(mylayers.SoftmaxLayer())
# ---Output Layer---
# fc_layer = mylayers.FullyConnectedLayer(20, 10)
# weights, biases = util.xaiver_init(20, 10)
# fc_layer.setWeights(weights)
# fc_layer.setBiases(biases)
# layers.append(fc_layer)
# layers.append(mylayers.SoftmaxLayer())
# ---Objective---
layers.append(mylayers.CrossEntropy())


### layers test

In [7]:
hyperparams = {"epoch": 500, "eta": 10e-8, "batch_size": 5, "rho1": 0.9, "rho2": 0.999}


In [8]:
%time
epoch = hyperparams['epoch']
eta = hyperparams['eta']
batch_size = hyperparams['batch_size']
rho1 = hyperparams['rho1']
rho2 = hyperparams['rho2']
ls_scores_train,Y_hat,ls_accuracy_train,ls_scores_valid,ls_accuracy_valid = test.adam_learn_with_validation(X_train,Y_train,X_valid,Y_valid,layers,mylayers.CrossEntropy,epoch=epoch,eta=eta,print_eval=False,print_layer=False,batch_size=batch_size)


Wall time: 0 ns


  7%|▋         | 33/500 [03:09<44:47,  5.75s/it]


KeyboardInterrupt: 

In [ ]:
train_results = {
    "ls_scores_train": ls_scores_train,
    "Y_hat": Y_hat,
    "ls_accuracy_train": ls_accuracy_train,
    "ls_scores_valid": ls_scores_valid,
    "ls_accuracy_valid": ls_accuracy_valid,
}


In [ ]:
# save result to pickle
with open("models/ANN_train_results_softmax.pkl", "wb") as f:
    pickle.dump(train_results, f)


In [ ]:
# read result from pickle
with open("models/ANN_train_results_softmax.pkl", "rb") as f:
    train_results = pickle.load(f)
ls_scores_train = train_results["ls_scores_train"]
Y_hat = train_results["Y_hat"]
ls_accuracy_train = train_results["ls_accuracy_train"]
ls_scores_valid = train_results["ls_scores_valid"]
ls_accuracy_valid = train_results["ls_accuracy_valid"]


In [ ]:
# graph of scores
hidden_layers = "softmax"
fig = util.draw_scores_with_validation(
    ls_scores_train, ls_scores_valid, hidden_layers, hyperparams
)
fig.show()


In [ ]:
# graph of accuracys
fig = util.draw_accuracys_with_validation(
    ls_accuracy_train, ls_accuracy_valid, hidden_layers, hyperparams
)
fig.show()


### validate accuracy

In [ ]:
labeled_Y_hat = encoder.decoding(Y_hat)
labeled_Y_hat


[7,
 3,
 8,
 9,
 3,
 9,
 7,
 7,
 5,
 4,
 2,
 5,
 6,
 8,
 1,
 7,
 4,
 9,
 0,
 7,
 9,
 3,
 1,
 0,
 6,
 1,
 3,
 9,
 3,
 6,
 0,
 2,
 4,
 9,
 0,
 1,
 8,
 7,
 8,
 8,
 0,
 1,
 2,
 0,
 6,
 5,
 5,
 7,
 2,
 0,
 1,
 3,
 9,
 3,
 0,
 6,
 2,
 2,
 6,
 4,
 0,
 1,
 8,
 4,
 0,
 2,
 6,
 7,
 7,
 9,
 4,
 0,
 8,
 4,
 8,
 6,
 0,
 0,
 0,
 9,
 6,
 3,
 6,
 7,
 9,
 4,
 8,
 6,
 6,
 1,
 9,
 5,
 2,
 6,
 3,
 4,
 9,
 1,
 0,
 4,
 2,
 8,
 5,
 4,
 4,
 4,
 4,
 8,
 2,
 4,
 5,
 4,
 5,
 1,
 3,
 3,
 4,
 2,
 1,
 1,
 2,
 1,
 8,
 2,
 7,
 0,
 4,
 0,
 6,
 3,
 5,
 2,
 2,
 5,
 1,
 1,
 4,
 5,
 2,
 2,
 9,
 3,
 1,
 2,
 9,
 9,
 5,
 3,
 4,
 8,
 7,
 5,
 3,
 7,
 9,
 9,
 0,
 1,
 8,
 0,
 5,
 6,
 0,
 3,
 8,
 0,
 6,
 9,
 1,
 8,
 3,
 5,
 2,
 9,
 3,
 5,
 5,
 1,
 2,
 1,
 0,
 1,
 7,
 8,
 8,
 1,
 5,
 8,
 9,
 9,
 8,
 8,
 0,
 1,
 5,
 7,
 4,
 3,
 7,
 4,
 4,
 4,
 1,
 9,
 2,
 2,
 1,
 5,
 0,
 4,
 6,
 5,
 1,
 6,
 6,
 8,
 3,
 2,
 2,
 4,
 2,
 8,
 1,
 6,
 0,
 2,
 9,
 4,
 0,
 5,
 3,
 4,
 3,
 3,
 6,
 7,
 2,
 7,
 0,
 9,
 2,
 0,
 8,
 0,
 1,
 3,
 9,
 0,
 8,
 7,


In [ ]:
labeled_Y = encoder.decoding(Y_train)
labeled_Y


[7,
 3,
 8,
 9,
 3,
 9,
 7,
 7,
 5,
 4,
 2,
 5,
 6,
 8,
 1,
 7,
 4,
 8,
 0,
 7,
 9,
 3,
 1,
 7,
 6,
 5,
 3,
 3,
 3,
 6,
 0,
 8,
 4,
 9,
 5,
 1,
 8,
 7,
 8,
 8,
 0,
 8,
 2,
 0,
 8,
 5,
 5,
 2,
 2,
 0,
 1,
 5,
 9,
 3,
 0,
 6,
 2,
 2,
 6,
 4,
 0,
 1,
 8,
 4,
 0,
 2,
 4,
 7,
 7,
 4,
 4,
 0,
 1,
 4,
 8,
 6,
 0,
 0,
 0,
 9,
 6,
 3,
 6,
 9,
 9,
 4,
 8,
 6,
 6,
 1,
 9,
 5,
 2,
 6,
 3,
 4,
 9,
 1,
 0,
 4,
 2,
 8,
 5,
 4,
 4,
 4,
 4,
 8,
 2,
 4,
 5,
 4,
 3,
 1,
 3,
 5,
 4,
 2,
 1,
 1,
 2,
 2,
 8,
 2,
 7,
 0,
 4,
 0,
 6,
 3,
 5,
 2,
 2,
 5,
 1,
 1,
 4,
 5,
 2,
 2,
 9,
 3,
 6,
 2,
 9,
 9,
 5,
 3,
 4,
 8,
 7,
 3,
 9,
 7,
 9,
 9,
 0,
 1,
 8,
 0,
 5,
 6,
 0,
 2,
 8,
 0,
 6,
 7,
 1,
 8,
 3,
 8,
 2,
 9,
 4,
 5,
 0,
 1,
 2,
 1,
 3,
 1,
 7,
 8,
 8,
 1,
 8,
 8,
 9,
 9,
 8,
 8,
 0,
 1,
 5,
 7,
 4,
 3,
 7,
 4,
 4,
 4,
 1,
 9,
 2,
 2,
 1,
 5,
 0,
 4,
 6,
 5,
 1,
 6,
 6,
 8,
 3,
 2,
 2,
 5,
 2,
 8,
 7,
 6,
 0,
 2,
 9,
 9,
 0,
 5,
 3,
 4,
 3,
 3,
 6,
 7,
 2,
 7,
 0,
 5,
 9,
 6,
 8,
 0,
 1,
 5,
 9,
 0,
 6,
 9,


In [ ]:
c = 0
for i in range(len(labeled_Y)):
    if labeled_Y[i] == labeled_Y_hat[i]:
        c += 1
print("Final Accuracy:", (c / len(labeled_Y)) * 100, "%")


Final Accuracy: 86.4 %
